In [1]:
import cv2
import os
import numpy as np

In [2]:
def cv_show(img, name='image'):
    cv2.imshow(name, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


In [3]:
def preprocessing_img(img):
    # 先padding再裁切
    
    out_w = 128
    out_h = 128
    top_size = int((128 - img.shape[0]) / 2)
    bottom_size = int((128 - img.shape[0]) / 2)
    dif_1 = 128 - (top_size + bottom_size + img.shape[0])
    top_size += dif_1
    left_size = int((128 - img.shape[1]) / 2)
    right_size = int((128 - img.shape[1]) / 2)
    dif_2 = 128 - (left_size + right_size + img.shape[1])
    left_size += dif_2    
    padding_v = tuple(img[-1, -1, :])
    if img.shape[1] <= 128:
        padding = cv2.copyMakeBorder(img, 
                                  top_size, bottom_size, left_size, right_size, 
                                  borderType=cv2.BORDER_CONSTANT, 
                                  value=(int(padding_v[0]), int(padding_v[1]), int(padding_v[2])))
    else:
        padding = cv2.copyMakeBorder(img, 
                          top_size, bottom_size, 0, 0, 
                          borderType=cv2.BORDER_CONSTANT, 
                          value=(int(padding_v[0]), int(padding_v[1]), int(padding_v[2])))
        padding = cv2.resize(padding, (128, 128))


    # 灰階 二值化
    gray = cv2.cvtColor(padding, cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(gray, 100, 255, cv2.THRESH_OTSU)
    # 黑白轉換
    thresh = 255 - thresh
    # 閉操作
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (9, 9))
    closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
    # 腐蝕 & 膨脹
    closed = cv2.erode(closed, None, iterations=1)
    closed = cv2.dilate(closed, None, iterations=1)
    # 輪廓檢測
    contours, hierarchy = cv2.findContours(closed, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    if len(contours) == 0:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        ret, thresh = cv2.threshold(gray, 220, 255, cv2.THRESH_BINARY)
        thresh = 255 - thresh

    else:    
        # 取出最大的
        c = sorted(contours, key=cv2.contourArea, reverse=True)[0]
        if cv2.contourArea(c) > 1000:
            # 閉操作
            kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
            closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
            # 腐蝕 & 膨脹
            closed = cv2.erode(closed, None, iterations=1)
            closed = cv2.dilate(closed, None, iterations=1)
            # 輪廓檢測
            contours, hierarchy = cv2.findContours(closed, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
            c = sorted(contours, key=cv2.contourArea, reverse=True)[0]
            
        rect = cv2.minAreaRect(c)
        box = np.int0(cv2.boxPoints(rect))
        # 裁切
        Xs = [i[0] for i in box]
        Ys = [i[1] for i in box]
        x1 = min(Xs)
        x2 = max(Xs)
        y1 = min(Ys)
        y2 = max(Ys)
        hight = y2 - y1
        width = x2 - x1
        padding = padding[y1: y1+hight, x1: x1+width]
        thresh = thresh[y1: y1+hight, x1: x1+width]
    # reshape
    img2 = cv2.resize(thresh, (64, 64))
    img2 = cv2.cvtColor(img2, cv2.COLOR_GRAY2BGR)
     
    return padding, img2

In [7]:
path = r'C:\Users\jason\Desktop\python\01_project\09_esun_2021\cv\cv\data-clean'
output_path =  r'C:\Users\jason\Desktop\python\01_project\09_esun_2021\cv\cv\test'

if not os.path.exists(f'{output_path}'):
    os.mkdir(f'{output_path}')

f_list = os.listdir(path)
for f in f_list:
    img_list = os.listdir(f'{path}/{f}')
    
    if not os.path.exists(f'{output_path}/{f}'): 
        os.mkdir(f'{output_path}/{f}')
    for idx in img_list:
        img = cv2.imread(f'{path}/{f}/{idx}')
        try:
            # 改function
            img2 = preprocessing_img(img)    
            cv2.imwrite(f'{output_path}/{f}/{idx}', img2)
        except Exception as e:
            print(f, idx, e)
        

# 維修區

In [8]:
f = 0
idx = '4618.jpg'


In [9]:
img = cv2.imread(f'{path}/{f}/{idx}')
_, img2 = preprocessing_img(img)   

In [13]:
cv_show(img)

In [12]:
# img = cv2.imread('test.jpg') 
# img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 
mask = np.zeros(img.shape[:2], np.uint8) 

bgdModel = np.zeros((1, 65), np.float64) 
fgdModel = np.zeros((1, 65), np.float64) 

rect = (2,2,630,930) 
cv2.grabCut(img,mask,rect,bgdModel,fgdModel,5,cv2.GC_INIT_WITH_RECT) 

mask2 = np.where((mask==2)|(mask==0), 0,1).astype('uint8') 
img = img*mask2[:,:, np.newaxis] 

In [10]:
cv_show(img2)

In [131]:
def padding_img(img):

    out_w = 128
    out_h = 128
    top_size = int((128 - img.shape[0]) / 2)
    bottom_size = int((128 - img.shape[0]) / 2)
    dif_1 = 128 - (top_size + bottom_size + img.shape[0])
    top_size += dif_1
    left_size = int((128 - img.shape[1]) / 2)
    right_size = int((128 - img.shape[1]) / 2)
    dif_2 = 128 - (left_size + right_size + img.shape[1])
    left_size += dif_2    
    padding_v = tuple(img[-1, -1, :])
    if img.shape[1] <= 128:
        padding = cv2.copyMakeBorder(img, 
                                  top_size, bottom_size, left_size, right_size, 
                                  borderType=cv2.BORDER_CONSTANT, 
                                  value=(255, 255, 255)
                        )
    else:
        padding = cv2.copyMakeBorder(img, 
                          top_size, bottom_size, 0, 0, 
                          borderType=cv2.BORDER_CONSTANT, 
                          value=(255, 255, 255)
                        )
        padding = cv2.resize(padding, (128, 128))
    return padding

In [153]:
def prepros_img(img):
    # 灰階 二值化
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(gray, 200, 255, cv2.THRESH_OTSU)
    # 黑白轉換
    thresh = 255 - thresh

    
    return thresh

In [84]:
def cut_img(img, kernel_size=(9, 9)):
    # 閉操作
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, kernel_size)
    closed = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)
    # 腐蝕 & 膨脹
    closed = cv2.erode(closed, None, iterations=1)
    closed = cv2.dilate(closed, None, iterations=1)
    
    contours, hierarchy = cv2.findContours(closed, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    tot_cnt = len(contours)
    c = sorted(contours, key=cv2.contourArea, reverse=True)[0]
    max_cnt = cv2.contourArea(c)
    rect = cv2.minAreaRect(c)
    box = np.int0(cv2.boxPoints(rect))
    # 裁切
    Xs = [i[0] for i in box]
    Ys = [i[1] for i in box]
    x1 = min(Xs)
    x2 = max(Xs)
    y1 = min(Ys)
    y2 = max(Ys)
    hight = y2 - y1
    width = x2 - x1
    img = img[y1: y1+hight, x1: x1+width]
    
    return img, tot_cnt, max_cnt

In [ ]:
# main

In [100]:
path = r'C:\Users\jason\Desktop\python\01_project\09_esun_2021\cv\cv\data-clean'


In [101]:
img_list = os.listdir(f'{path}/0')

In [159]:
for img_path in img_list:
    print('=================')
    img = cv2.imread(f'{path}/0/{img_path}')
    print(img_path)
    print(img.shape)
    cv_show(img, name=img_path)
    thresh = prepros_img(img)
    cut, tot_cnt, max_cnt = cut_img(thresh)
    

    if  max_cnt > 1100:
        print(1)

        padding = padding_img(img)
        thresh = prepros_img(padding)
        cut, tot_cnt, max_cnt = cut_img(thresh)
        final = cv2.resize(cut, (128, 128))
        cv_show(final, name=img_path) 
    elif tot_cnt <= 1:
        print(2)        

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        ret, thresh = cv2.threshold(gray, 220, 255, cv2.THRESH_OTSU)
        final = 255 - thresh
        final = cv2.resize(final, (128, 128))
        cv_show(final, name=img_path)
    else:
        print(3)
        final = cv2.resize(cut, (128, 128))
        cv_show(final, name=img_path)   
    print('=================')


1027.jpg
(67, 40, 3)
2
10507.jpg
(67, 39, 3)
3
10692.jpg
(67, 70, 3)
1
11016.jpg
(67, 70, 3)
1
11417.jpg
(67, 35, 3)
2
11477.jpg
(67, 56, 3)
2
11882.jpg
(67, 59, 3)
2
13167.jpg
(67, 74, 3)
1
1324.jpg
(67, 63, 3)
3
13883.jpg
(67, 53, 3)
3
14375.jpg
(67, 65, 3)
1
15993.jpg
(67, 71, 3)
3
16786.jpg
(67, 45, 3)
2
17361.jpg
(67, 45, 3)
3
1738.jpg
(67, 43, 3)
3
17534.jpg
(67, 29, 3)
2
19615.jpg
(67, 61, 3)
1
19756.jpg
(67, 61, 3)
3
19759.jpg
(67, 59, 3)
3
20790.jpg
(67, 45, 3)
2
21077.jpg
(67, 83, 3)
1
2150.jpg
(67, 43, 3)
2
21942.jpg
(67, 107, 3)
3
22144.jpg
(67, 65, 3)
3
22683.jpg
(67, 45, 3)
1
23379.jpg
(67, 57, 3)
1
24185.jpg
(67, 67, 3)
1
24286.jpg
(67, 56, 3)
3
25284.jpg
(67, 64, 3)
3
27784.jpg
(67, 59, 3)
3
2974.jpg
(67, 52, 3)
3
30045.jpg
(67, 66, 3)
1
31393.jpg
(67, 54, 3)
1
32169.jpg
(67, 57, 3)
1
32433.jpg
(67, 35, 3)
2
32622.jpg
(67, 44, 3)
1
33878.jpg
(67, 51, 3)
2
34341.jpg
(67, 66, 3)
3
34965.jpg
(67, 62, 3)
1
35909.jpg
(67, 59, 3)
1
37752.jpg
(67, 54, 3)
1
38058.jpg
(67, 42, 3

error: OpenCV(4.5.1) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-oduouqig\opencv\modules\imgproc\src\resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


In [ ]:
11417
11882

In [116]:
img_list[:10]

['1027.jpg',
 '10507.jpg',
 '10692.jpg',
 '11016.jpg',
 '11417.jpg',
 '11477.jpg',
 '11882.jpg',
 '13167.jpg',
 '1324.jpg',
 '13883.jpg']

In [157]:
img_path = img_list[6]
img = cv2.imread(f'{path}/0/{img_path}')
cv_show(img)
thresh = prepros_img(img)
cv_show(thresh)
cut, tot_cnt, max_cnt = cut_img(thresh)
print(tot_cnt, max_cnt)
cv_show(cut)


1 1039.0


In [155]:
img.shape

(67, 59, 3)

In [156]:
# if (max_cnt > 1000):
padding = padding_img(img)
cv_show(padding)
thresh = prepros_img(padding)
cv_show(thresh)

cut, tot_cnt, max_cnt = cut_img(thresh)
final = cv2.resize(cut, (128, 128))
cv_show(final, name=img_path) 

In [122]:
cv2.contourArea(cut)

error: OpenCV(4.5.1) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-oduouqig\opencv\modules\imgproc\src\shapedescr.cpp:315: error: (-215:Assertion failed) npoints >= 0 && (depth == CV_32F || depth == CV_32S) in function 'cv::contourArea'


In [ ]:
if tot_cnt == 0:
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(gray, 220, 255, cv2.THRESH_BINARY)
    final = 255 - thresh
    cv_show(final, name=img_path)
else:
    final = cv2.resize(cut, (128, 128))
    cv_show(final, name=img_path)

In [110]:
cv_show(thresh)

In [118]:
# 檢測區
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (9, 9))
closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

# 腐蝕 & 膨脹
closed = cv2.erode(closed, None, iterations=1)
closed = cv2.dilate(closed, None, iterations=1)

contours, hierarchy = cv2.findContours(closed, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
tot_cnt = len(contours)
c = sorted(contours, key=cv2.contourArea, reverse=True)[0]
max_cnt = cv2.contourArea(c)
rect = cv2.minAreaRect(c)
box = np.int0(cv2.boxPoints(rect))

In [ ]:
# 裁切
Xs = [i[0] for i in box]
Ys = [i[1] for i in box]
x1 = min(Xs)
x2 = max(Xs)
y1 = min(Ys)
y2 = max(Ys)
hight = y2 - y1
width = x2 - x1
img = img[y1: y1+hight, x1: x1+width]

In [119]:
# 畫圖區
draw_img = img.copy()
res = cv2.drawContours(draw_img, contours, -1, (0, 0, 255), 2) # 繪製圖像, 輪廓, 輪廓索引, 顏色模式(B, G, R), 線條厚度
cv_show(res)

In [120]:
draw_img = img.copy()
for c in contours:
    rect = cv2.minAreaRect(c)
    box = np.int0(cv2.boxPoints(rect))
    res = cv2.drawContours(draw_img, [box], -1, (0, 255, 0), 3)
cv_show(res)

In [ ]:
def cut_img(img):
    # 輪廓檢測
    contours, hierarchy = cv2.findContours(closed, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    if len(contours) == 0:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        ret, thresh = cv2.threshold(gray, 220, 255, cv2.THRESH_BINARY)
        thresh = 255 - thresh

    else:    
        # 取出最大的
        c = sorted(contours, key=cv2.contourArea, reverse=True)[0]
        if cv2.contourArea(c) > 1000:
            # 閉操作
            kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
            closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
            # 腐蝕 & 膨脹
            closed = cv2.erode(closed, None, iterations=1)
            closed = cv2.dilate(closed, None, iterations=1)
            # 輪廓檢測
            contours, hierarchy = cv2.findContours(closed, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
            c = sorted(contours, key=cv2.contourArea, reverse=True)[0]
            
        rect = cv2.minAreaRect(c)
        box = np.int0(cv2.boxPoints(rect))
        # 裁切
        Xs = [i[0] for i in box]
        Ys = [i[1] for i in box]
        x1 = min(Xs)
        x2 = max(Xs)
        y1 = min(Ys)
        y2 = max(Ys)
        hight = y2 - y1
        width = x2 - x1
        padding = padding[y1: y1+hight, x1: x1+width]
        thresh = thresh[y1: y1+hight, x1: x1+width]